In [ ]:
!pip install pykan

In [ ]:
!pip show pykan

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from kan import KAN
import uuid

# Установка сида для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def true_function(X):
    x1, x2, x3, x4, x5 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]
    part1 = np.sin(np.exp(x1 + x2))
    part2 = np.log(1 + x3**2 + np.cos(x4))
    part3 = - np.sqrt(np.abs(x5))
    return part1 + part2 + part3

In [ ]:
N_train = 5000
N_test = 1000

x_train = np.random.uniform(-2, 2, size=(N_train, 5))
y_train = true_function(x_train).reshape(-1, 1)
x_test = np.random.uniform(-2, 2, size=(N_test, 5))
y_test = true_function(x_test).reshape(-1, 1)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)


In [ ]:
class MLP(nn.Module):
    def __init__(self, layers):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
            if i < len(layers) - 2:
                self.layers.append(nn.ReLU())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
def train_mlp(model, x, y, epochs=1000, lr=0.01):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    loss_history = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
    return model, loss_history

In [ ]:
mlp_configs = [
    {"layers": [5, 50, 50, 1], "epochs": 2000, "lr": 0.001, "name": "MLP Medium"},
    {"layers": [5, 100, 100, 100, 1], "epochs": 3000, "lr": 0.0005, "name": "MLP Deep"},
]

mlp_models = []
mlp_predictions = []
mlp_mse = []
mlp_loss_histories = []

for config in mlp_configs:
    mlp_model = MLP(config["layers"])
    mlp_model, loss_history = train_mlp(mlp_model, x_train_tensor, y_train_tensor, config["epochs"], config["lr"])
    mlp_models.append(mlp_model)
    mlp_loss_histories.append(loss_history)
    with torch.no_grad():
        y_pred = mlp_model(x_test_tensor).cpu().numpy()
        mlp_predictions.append(y_pred)
        mse = np.mean((y_pred - y_test)**2)
        mlp_mse.append(mse)

In [ ]:
kan_configs = [
    {"width": [5, 5, 1], "grid": 15, "k": 3, "name": "KAN Small"},
    {"width": [5, 10, 1], "grid": 15, "k": 3, "name": "KAN Medium"},
]

kan_models = []
kan_predictions = []
kan_mse = []
kan_loss_histories = []

for config in kan_configs:
    kan_model = KAN(width=config["width"], grid=config["grid"], k=config["k"], device=device)
    dataset = {
        "train_input": x_train_tensor,
        "train_label": y_train_tensor,
        "test_input": x_test_tensor,
        "test_label": y_test_tensor
    }
    results = kan_model.fit(dataset, opt="LBFGS", steps=300, lamb=0.01)
    kan_model(dataset['train_input'])
    kan_models.append(kan_model)
    y_pred = kan_model(x_test_tensor).detach().cpu().numpy()
    kan_predictions.append(y_pred)
    mse = np.mean((y_pred - y_test)**2)
    kan_mse.append(mse)
    kan_loss_histories.append(results['train_loss'])

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
for i, loss_history in enumerate(mlp_loss_histories):
    plt.plot(range(len(loss_history)), loss_history, label=mlp_configs[i]['name'])
plt.title("MLP Loss Curves")
plt.xlabel("Epoch")
plt.ylabel("Train MSE")
plt.legend()
plt.grid(True)
plt.yscale("log")

plt.subplot(1, 2, 2)
for i, loss_history in enumerate(kan_loss_histories):
    plt.plot(range(len(loss_history)), loss_history, label=kan_configs[i]['name'])
plt.title("KAN Loss Curves")
plt.xlabel("Step")
plt.ylabel("Train MSE")
plt.legend()
plt.grid(True)
plt.yscale("log")

plt.tight_layout()
plt.savefig("mlp_kan_compositional_5d.png")

In [ ]:
plt.figure(figsize=(10, 6))

# Для MLP рисуем по эпохам
for i, loss_history in enumerate(mlp_loss_histories):
    plt.plot(range(len(loss_history)), loss_history, label=mlp_configs[i]['name'] + " (MLP)")

# Для KAN рисуем по шагам
for i, loss_history in enumerate(kan_loss_histories):
    plt.plot(range(len(loss_history)), loss_history, label=kan_configs[i]['name'] + " (KAN)")

plt.title("Сравнение обучения MLP и KAN (Train MSE)")
plt.xlabel("Эпоха / Шаг")
plt.ylabel("Ошибка (MSE)")
plt.yscale("log")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("mlp_kan_comparison_loss_curve.png")
plt.show()

In [ ]:
mode = "auto" # "manual"

if mode == "manual":
    # manual mode
    kan_model.fix_symbolic(0,0,0,'sin');
    kan_model.fix_symbolic(0,1,0,'x^2');
    kan_model.fix_symbolic(1,0,0,'exp');
elif mode == "auto":
    # automatic mode
    lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs', 'cos']
    kan_model.auto_symbolic(lib=lib)


In [ ]:
kan_model.symbolic_formula()[0][0]